# Stanford Campus Speech

This notebook is an example of using the OpenAlex API to explore what researchers at Stanford University have written about *campus speech*, *free speech* and *academic freedom*.

## The OpenAlex API

First lets see how many articles there are from Stanford University in OpenAlex.

In [1]:
import pyalex

pyalex.config.email = "ehs@pobox.com"
pyalex.config.retry_http_codes = [429, 500, 503]

pyalex.Works().filter(institutions={"id": "I97018004"}).count()

445835

We can also search the title and abstract for Works that match the phrase "Academic Freedom".

In [2]:
pyalex.Works().filter(title_and_abstract={"search": "'Academic Freedom'"}).count()

28219

Here is what the titles of the first 10 results looks like:

In [3]:
pyalex.Works().filter(title_and_abstract={"search": '"Academic Freedom"'}).select(["title", "publication_year"]).get(per_page=10)

[{'title': 'Academic freedom, private‐sector focus, and the process of innovation',
  'publication_year': 2008},
 {'title': 'The Development of Academic Freedom in the United States',
  'publication_year': 1956},
 {'title': 'Academic freedom, autonomy and accountability in British universities',
  'publication_year': 1990},
 {'title': 'Academic Freedom Under Fire: The Case for Critical Pedagogy',
  'publication_year': 2006},
 {'title': 'The Future of Academic Freedom', 'publication_year': 1997},
 {'title': 'Academic freedom in the age of the university',
  'publication_year': 1961},
 {'title': 'Academic Freedom in Clinical Research', 'publication_year': 2002},
 {'title': 'Academic Freedom in Europe: A Preliminary Comparative Analysis',
  'publication_year': 2007},
 {'title': 'Unintended consequences of cooperative research: impact of industry sponsorship on climate for academic freedom and other graduate student outcome',
  'publication_year': 2001},
 {'title': 'The 1940 Statement of P

The search can be composed with a filter to limit to Works where one of the authors is from Stanford:

In [4]:
pyalex.Works().search('"Academic Freedom"').filter(institutions={"id": "I97018004"}).select(["title"]).get(per_page=10)

[{'title': 'Doing Medical Journals Differently: <i>Open Medicine,</i> Open Access, and Academic Freedom'},
 {'title': 'The Perversion of Virtue―Causes and Consequences of Threats to Academic Freedom in the Contemporary University'},
 {'title': 'Education in Safe and Unsafe Spaces'},
 {'title': 'Illiberal Reactions to Higher Education'},
 {'title': 'Academic Freedom in America — In Support of Institutional Voices'},
 {'title': 'The formalization of the university: rules, roots, and routes'},
 {'title': 'Policies on Faculty Conflicts of Interest at US Universities'},
 {'title': 'Adaptations of Avian Flu Virus Are a Cause for Concern'},
 {'title': 'A New Model for Industry–Academic Partnerships'},
 {'title': 'Eyes Wide Shut: University, State and Society'}]

To determine who the Stanford researchers are we need to request the `authorships` data and filter where the `I97018004` code is listed as one of their institutions.

In [5]:
work = pyalex.Works().search('"Academic Freedom"').filter(institutions={"id": "I97018004"}).select(["title", "authorships"]).get(per_page=1)[0]
work

{'title': 'Doing Medical Journals Differently: <i>Open Medicine,</i> Open Access, and Academic Freedom',
 'authorships': [{'author_position': 'first',
   'author': {'id': 'https://openalex.org/A5008862123',
    'display_name': 'John Willinsky',
    'orcid': 'https://orcid.org/0000-0001-6192-8687'},
   'institutions': [{'id': 'https://openalex.org/I141945490',
     'display_name': 'University of British Columbia',
     'ror': 'https://ror.org/03rmrcq20',
     'country_code': 'CA',
     'type': 'education',
     'lineage': ['https://openalex.org/I141945490']},
    {'id': 'https://openalex.org/I97018004',
     'display_name': 'Stanford University',
     'ror': 'https://ror.org/00f54p054',
     'country_code': 'US',
     'type': 'education',
     'lineage': ['https://openalex.org/I97018004']}],
   'countries': ['CA', 'US'],
   'is_corresponding': False,
   'raw_author_name': 'John Willinsky',
   'raw_affiliation_strings': ['Stanford University and the University of British Columbia'],
   '

In [6]:
for author in work['authorships']:
    if 'https://openalex.org/I97018004' in [i['id'] for i in author['institutions']]:
        print(author['author']['display_name'])

John Willinsky


## Putting it Together

Lets create a function that does this search and returns an iterator where each result is a tuple of an author and a work. One thing that is also useful to do here is limit to publications from the last 10 years since we are trying to find researchers who are currently at Stanford.

In [7]:
def search(q):
    for page in pyalex.Works().search(f"{q}").filter(institutions={"id": "I97018004"}).paginate(per_page=200):
        for work in page:
            for author in work['authorships']:
                if work['publication_year'] >= 2014 and 'https://openalex.org/I97018004' in [i['id'] for i in author['institutions']]:
                    yield (author, work)

author, work = next(search('"academic freedom"'))
print(author['author']['display_name'])
print(work['title'])
print(work['id'])

Erika T. Hermanowicz
The Perversion of Virtue―Causes and Consequences of Threats to Academic Freedom in the Contemporary University
https://openalex.org/W4367319927


It would be useful to have a function that does the query and summarizes the results by printing the research name and the list of work IDs that matched:

In [30]:
from collections import defaultdict
from IPython.display import display, Markdown

def print_results(q):
    results = defaultdict(list)
    
    for author, work in search(q):
        results[author['author']['display_name']].append(work)

    for name in sorted(results, key=lambda name: len(results[name]), reverse=True):
        lines = [f"* {name}"]
        for work in results[name]:
            lines.append(f"  * [{work['title']}]({work['id']})")
        display(Markdown("\n".join(lines)))


### Academic Freedom

In [31]:
print_results('"academic freedom"')

* Michelle M. Mello
  * [Beyond financial conflicts of interest: Institutional oversight of faculty consulting agreements at schools of medicine and public health](https://openalex.org/W2949573220)
  * [Population health science as a unifying foundation for translational clinical and public health research](https://openalex.org/W4214523602)
  * [Beyond financial conflicts of interest: Institutional oversight of faculty consulting agreements at schools of medicine and public health](https://openalex.org/W2886563298)

* John W. Meyer
  * [Illiberal Reactions to Higher Education](https://openalex.org/W4289866241)
  * [Global Neoliberalism as a Cultural Order and Its Expansive Educational Effects](https://openalex.org/W4205238454)

* Francisco O. Ramírez
  * [Beyond internationalisation and isomorphism – the construction of a global higher education regime](https://openalex.org/W2962437407)
  * [Reputation management by Chinese universities: Primary profile and comparative features](https://openalex.org/W3027627788)

* John P. A. Ioannidis
  * [Scientific petitions and open letters in the era of covid-19](https://openalex.org/W3093968437)
  * [Conflict of Interest in Nutrition Research—Reply](https://openalex.org/W2810885292)

* Jay Bhattacharya
  * [Stagnation and Scientific Incentives](https://openalex.org/W3006456429)
  * [Dr. Jay Bhattacharya Reveals Stanford University’s Attempts To Derail COVID Studies](https://openalex.org/W4386924307)

...

* Ayça Alemdaroğlu
  * [The University in the Making of Authoritarian Turkey](https://openalex.org/W4383960951)

* Cassie A. Wright
  * [<i>Composition in the Age of Austerity</i>, Nancy Welch and Tony Scott, eds.](https://openalex.org/W2613122352)

* Chris Ford
  * [Processing](https://openalex.org/W4250431239)

* Jens Jungblut
  * [A look back on some books we did not manage to review in 2017](https://openalex.org/W2783807313)

* John Willinsky
  * [If, Alongside Libraries, Funders Pulled Their Weight: A Study in Universal Open Access](https://openalex.org/W2957028843)

### Free Speech

In [32]:
print_results('"free speech"')

* Josiah Ober
  * [Development and Political Theory in Classical Athens](https://openalex.org/W3122719624)
  * [Development and Political Theory in Classical Athens](https://openalex.org/W1095857336)
  * [A Theory of Democracy](https://openalex.org/W2738706274)
  * [Democracy after Liberalism](https://openalex.org/W2738337334)

* Barry R. Weingast
  * [Development and Political Theory in Classical Athens](https://openalex.org/W3122719624)
  * [Development and Political Theory in Classical Athens](https://openalex.org/W1095857336)
  * [Capitalism, Democracy, and Countermajoritarian Institutions](https://openalex.org/W1059681994)

* Hakeem Jefferson
  * [Beyond the Ballot Box: A Conversation About Democracy and Policing in the United States](https://openalex.org/W4297519268)
  * [White racial identity, political attitudes, and selective exposure to information about racism](https://openalex.org/W4379207953)
  * [None](https://openalex.org/W4297509984)

* Anna D. Gibson
  * [Free Speech and Safe Spaces: How Moderation Policies Shape Online Discussion Spaces](https://openalex.org/W2932595524)
  * [Safe Spaces &amp; Free Speech: Effects of Moderation Policy on Structures of Online Forum Discussions](https://openalex.org/W2574524142)

* Eamonn Callan
  * [Education in Safe and Unsafe Spaces](https://openalex.org/W2566803539)
  * [Debate: Liberal Virtues and Civic Education](https://openalex.org/W1914456685)

...

* Ted Berrigan
  * [Literature of the Americas](https://openalex.org/W4285060219)

* Bernadette But
  * [Literature of the Americas](https://openalex.org/W4285060219)

* New York
  * [Literature of the Americas](https://openalex.org/W4285060219)

* Анна Швец
  * [Literature of the Americas](https://openalex.org/W4285060219)

* Ben A. Barres
  * [Ben A. Barres](https://openalex.org/W4324263578)

### Campus Free Speech

In [34]:
print_results('campus "free speech"')

* Hakeem Jefferson
  * [Beyond the Ballot Box: A Conversation About Democracy and Policing in the United States](https://openalex.org/W4297519268)
  * [None](https://openalex.org/W4297509984)
  * [White racial identity, political attitudes, and selective exposure to information about racism](https://openalex.org/W4379207953)

* Tejúmólá Ọláníyan
  * [Thinking with an Accent](https://openalex.org/W4366088804)
  * [Thinking with an Accent: Toward a New Object, Method, and Practice](https://openalex.org/W4321435131)

* Emily Apter
  * [Thinking with an Accent](https://openalex.org/W4366088804)
  * [Thinking with an Accent: Toward a New Object, Method, and Practice](https://openalex.org/W4321435131)

* Lital Levy
  * [Thinking with an Accent](https://openalex.org/W4366088804)
  * [Thinking with an Accent: Toward a New Object, Method, and Practice](https://openalex.org/W4321435131)

* Vicente L. Rafael
  * [Thinking with an Accent](https://openalex.org/W4366088804)
  * [Thinking with an Accent: Toward a New Object, Method, and Practice](https://openalex.org/W4321435131)

...

* Michael Winton
  * [None](https://openalex.org/W4297509984)

* Green Distin
  * [None](https://openalex.org/W4297509984)

* Ayça Alemdaroğlu
  * [The University in the Making of Authoritarian Turkey](https://openalex.org/W4383960951)

* Jay Bhattacharya
  * [Dr. Jay Bhattacharya Reveals Stanford University’s Attempts To Derail COVID Studies](https://openalex.org/W4386924307)

* Ben A. Barres
  * [Ben A. Barres](https://openalex.org/W4324263578)

In [35]:
print_results('"Kalven Report"')

* Robin Jia
  * [Adversarial Examples for Evaluating Reading Comprehension Systems](https://openalex.org/W2963969878)

* Percy Liang
  * [Adversarial Examples for Evaluating Reading Comprehension Systems](https://openalex.org/W2963969878)

In [36]:
print_results('campus civility')

* Eamonn Callan
  * [Education in Safe and Unsafe Spaces](https://openalex.org/W2566803539)

* David Y. Yang
  * [Curriculum and Ideology](https://openalex.org/W4214683418)

* Nel Noddings
  * [The Search for Meaning and Connection](https://openalex.org/W2573326911)

* Erika T. Hermanowicz
  * [The Perversion of Virtue―Causes and Consequences of Threats to Academic Freedom in the Contemporary University](https://openalex.org/W4367319927)

* Joseph C. Hermanowicz
  * [The Perversion of Virtue―Causes and Consequences of Threats to Academic Freedom in the Contemporary University](https://openalex.org/W4367319927)

* James H. Mittelman
  * [A Better Intellectual Community Is Possible: Dialogues with Ali A. Mazrui](https://openalex.org/W2060377431)

* Jisha Menon
  * [Performing the Secular](https://openalex.org/W2756119157)

* Lee Ross
  * [Barriers to agreement in the asymmetric Israeli–Palestinian conflict<sup>1</sup>](https://openalex.org/W2038228457)